# Continuous Control

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the second project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Reacher.app"`
- **Windows** (x86): `"path/to/Reacher_Windows_x86/Reacher.exe"`
- **Windows** (x86_64): `"path/to/Reacher_Windows_x86_64/Reacher.exe"`
- **Linux** (x86): `"path/to/Reacher_Linux/Reacher.x86"`
- **Linux** (x86_64): `"path/to/Reacher_Linux/Reacher.x86_64"`
- **Linux** (x86, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86"`
- **Linux** (x86_64, headless): `"path/to/Reacher_Linux_NoVis/Reacher.x86_64"`

For instance, if you are using a Mac, then you downloaded `Reacher.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Reacher.app")
```

In [ ]:
env = UnityEnvironment(file_name='../../Reacher.app')

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, a double-jointed arm can move to target locations. A reward of `+0.1` is provided for each step that the agent's hand is in the goal location. Thus, the goal of your agent is to maintain its position at the target location for as many time steps as possible.

The observation space consists of `33` variables corresponding to position, rotation, velocity, and angular velocities of the arm.  Each action is a vector with four numbers, corresponding to torque applicable to two joints.  Every entry in the action vector must be a number between `-1` and `1`.

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]     # reset the environment    

states = env_info.vector_observations                  # get the current state (for each agent)
#print(states, type(states))
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
#print(scores)
cnt = 0
#agent = ActorCritic(state_size, action_size).to(device)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    #dist, value = agent(torch.Tensor(states).to(device)) 
    #actions = dist.sample()
    print(actions)
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
#    print(env_info)
    next_states = env_info.vector_observations         # get next state (for each agent)
#    print(next_states)
    rewards = env_info.rewards                         # get reward (for each agent)
#    print(rewards)
    dones = env_info.local_done                        # see if episode finished
#    print(dones)
    scores += env_info.rewards                         # update the score (for each agent)
    print(scores)
    states = next_states                               # roll over states to next time step
    
    cnt+=1
    if np.any(dones):                                  # exit loop if episode finished
        break
    
print('Total score (averaged over agents) this episode: {} in time:{}'.format(np.mean(scores), cnt))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

### 1.Install packages

In [ ]:
from unityagents import UnityEnvironment
import numpy as np

In [ ]:
import warnings

warnings.filterwarnings("ignore")
#import gym
#import pybullet_envs
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
#import torch.multiprocessing as mp
import time
import random

from collections import deque
from statistics import mean, stdev

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")
use_cuda

### 2.Examine the State and Action Spaces

In [ ]:
env = UnityEnvironment(file_name='../../Reacher.app')

brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 3. Define Network

In [ ]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)
        
class ActorCritic(nn.Module):
    actor = None
    actor_optimizer = None
    critic = None
    critic_optimizer = None
    
    def __init__(self, state_size, action_size, hidden_size=256, seed=100, std=0.0):
        super(ActorCritic, self).__init__()
        self.seed = torch.manual_seed(seed)
        
        if Agent.actor is None:
            self.critic = nn.Sequential(
                nn.Linear(state_size, 256),
                nn.LeakyReLU(),
                nn.Linear(256, 128),
                nn.LeakyReLU(),
                nn.Linear(128, 128),
                nn.LeakyReLU(),
                nn.Linear(128, 64),
                nn.LeakyReLU(),
                nn.Linear(64, 1)
            )
            
        if Agent.critic is None:
            self.actor = nn.Sequential(
                nn.Linear(state_size, 256),
                nn.ReLU(),
                nn.Linear(256, 128),
                nn.ReLU(),
                nn.Linear(128, action_size),
                nn.Tanh()
            )
        
        self.actor = Agent.actor
        self.actor_optimizer = Agent.actor_optimizer
        self.critic = Agent.critic
        self.critic_optimizer = Agent.critic_optimizer

        self.log_std = nn.Parameter(torch.ones(1, action_size) * std)

        self.apply(init_weights)
        
    def forward(self, x):
        self.value = self.critic(x)
        self.mu    = self.actor(x)      
        self.std   = self.log_std.exp().expand_as(self.mu)
        self.dist  = Normal(self.mu, self.std)
        return self.dist, self.value
    
    def act(self, x):
        self.mu    = self.actor(x)      
        self.std   = self.log_std.exp().expand_as(self.mu)
        self.dist  = Normal(self.mu, self.std)
        return dist.sample()
        
    def learn(self, state_lst, logprob_lst, q_val_lst, entropy, optimizer):
        
        self.log_probs = torch.cat(logprob_lst)
        self.returns   = torch.cat(q_val_lst).detach()
        self.values    = torch.cat(state_lst)
        
        self.advantage = self.returns - self.values
              
        self.critic_loss = self.advantage.pow(2).mean() 
        
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic_local.parameters(), 1)
        self.critic_optimizer.step()
        
        self.actor_loss  = -(self.log_probs * self.advantage.detach()).mean()
        
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        #self.loss = self.actor_loss + 0.5 * self.critic_loss - 0.001 * entropy
        #optimizer.zero_grad()
        #self.loss.backward()    
        #optimizer.step()


### 4.Test using random action

In [ ]:
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

In [ ]:
def compute_returns(next_value, rewards, masks, gamma=0.99):
    R = next_value
    returns = []
    for step in reversed(range(len(rewards))):
        R = rewards[step] + gamma * R * masks[step]
        returns.insert(0, R)
    return returns

def compute_gae(next_value, rewards, masks, values, gamma=0.99, tau=0.95):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

In [ ]:
n_step = 20
num_episodes = 2000

agent = ActorCritic(state_size, action_size).to(device)
optimizer = optim.Adam(agent.parameters(), lr=1e-4) 
    
##########################이 부분은 수정하지 마세요###########################
episode_rewards = deque(maxlen=100)
start_time = time.time()
epi_plot = []
finish = False
##########################이 부분은 수정하지 마세요###########################

agents =[] 

for i in range(num_agents):
    agents.append(Agent(state_size, action_size, random_seed=0))

for episode in range(num_episodes):
    
    env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
    states = env_info.vector_observations                  # get the current state (for each agent)
    scores = np.zeros(num_agents)                          # initialize the score (for each agent)
    dones = np.zeros(num_agents, dtype=bool)
    
    for agent in agents:
        agent.reset()
    
    while not np.any(dones):
        
        log_probs = []
        values    = []
        nstep_rewards   = []
        masks     = []
        entropy = 0
        
        for step in range(n_step):
            actions = [agents[i].act(states[i]) for i in range(num_agents)]
            #dist, value = agent(torch.Tensor(states).to(device)) # agent returns dist, value in tensor seq type
            #actions = dist.sample()                              # .sample() returns actions in tensor seq type

            env_info = env.step(actions.numpy())[brain_name]     # env_info.vector_observations are in numpy type
            next_states = env_info.vector_observations           # 
            rewards = env_info.rewards                           # env_info.rewards, local_done are in list type
            dones = np.array(env_info.local_done)                # local_done returns list type, convert to numpy!
            scores += env_info.rewards                           # scores defined as numpy, 
            states = next_states                                 # roll over states to next time step
            
            log_prob = dist.log_prob(actions)
            entropy += dist.entropy().mean()                     # by .mean(), entropy is in tensor const. type
            
            log_probs.append(log_prob)
            values.append(value)
            nstep_rewards.append(torch.FloatTensor(rewards).unsqueeze(1).to(device))
            masks.append(torch.FloatTensor(1 - dones).unsqueeze(1).to(device))

            if np.any(dones):                                  # exit loop if episode finished
                break
        
        _, next_values = agent(torch.FloatTensor(next_states).to(device))
        next_values = torch.zeros(num_agents).unsqueeze(1) if np.any(dones) else next_values

        #returns = compute_returns(next_values, nstep_rewards, masks)
        returns =  compute_gae(next_values, nstep_rewards, masks, values)
        
        agent.learn(values, log_probs, returns, entropy, optimizer)
    
    episode_rewards.append(np.mean(scores))

    #print('\rEpisode {}\tScore {:.2f}\tAverage Score: {:.2f}'.format(episode, np.mean(scores), np.mean(episode_rewards)), end="")

    if episode >= 10:
        mean_100_episode_reward = np.mean(episode_rewards)
        epi_plot.append(mean_100_episode_reward)
        if episode % 10 == 0:
            print("\nEpisode: {}, scores: {} scores.avg: {:.1f}".format(episode, np.mean(scores), mean_100_episode_reward))

        if mean_100_episode_reward >= 30:
            finish = True
            print("Solved (1)!!!, Time : {:.2f}".format(time.time() - start_time))
            np.save("./single.npy", np.array(epi_plot))
            if not os.path.isdir(SAVE_PATH):
                os.mkdir(SAVE_PATH)
            torch.save(agent.state_dict(),SAVE_PATH+'checkpoint_actor.pth')
            break 
            
env.close()
print("Fail... Retry")


In [ ]:
env.close()

In [ ]:
scores = Agent()

fig = plt.figure()
ax = fig.add_subplot(111)
ax.axhline(y=30, xmin=0.0, xmax=1.0, color='r', linestyle='--', linewidth=0.7, alpha=0.9)
plt.plot(np.arange(1, len(epi_plot)+1), epi_plot)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

In [ ]:
agent.load_state_dict(torch.load('checkpoint.pth'))

while True:
    with torch.no_grad():
        dist, value = agent(torch.Tensor(states).to(device)) 
        actions = dist.sample()
                
    env_infos = env.step(actions.numpy())[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break

#print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

In [ ]:
env.close()